In [ ]:
import os
import matplotlib.image as mpimg
from PIL import Image
import tensorflow as tf
from helpers import *
from constants import *
from NNmodels import *
import tensorflow as tf
from tensorflow.keras import  callbacks
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import os



%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data extraction and splitting

In [ ]:

data_dir = os.getcwd() + '/dataset/training/'
data_filename = data_dir + "images/"
labels_filename = data_dir + "groundtruth/"


data, labels = load_data(data_filename, labels_filename, TRAINING_SIZE)


Loading 100 images
[[0.3254902  0.3019608  0.27058825]
 [0.31764707 0.28627452 0.25490198]
 [0.3137255  0.2901961  0.25490198]
 ...
 [0.31764707 0.31764707 0.29411766]
 [0.3137255  0.3137255  0.2901961 ]
 [0.31764707 0.31764707 0.3019608 ]]
Loading 100 images


In [5]:

train_split = 0.7
val_split = 0.15
test_split = 0.15

train_data = data[:int(train_split * len(data))]
train_labels = labels[:int(train_split * len(labels))]
val_data = data[int(train_split * len(data)) : int((train_split + val_split) * len(data))]
val_labels = labels[int(train_split * len(labels)) : int((train_split + val_split) * len(labels))]
test_data = data[int((train_split + val_split) * len(data)):]
test_labels = labels[int((train_split + val_split) * len(labels)):]


# Model definition and training

In [ ]:
model = CNN(input_shape=(100, 100, 3), num_classes=2)
model.summary()


In [21]:
def create_generators(train_data, train_labels, val_data, val_labels, test_data, test_labels, WINDOW_SIZE, batch_size=32):

    c_weights = {0: 2.8, 1: 1}

    train_generator = image_generator(train_data,train_labels,WINDOW_SIZE,batch_size=batch_size,class_weights = c_weights)

    val_generator = image_generator(val_data,val_labels,WINDOW_SIZE,batch_size=batch_size)

    test_generator = image_generator(test_data,test_labels,WINDOW_SIZE,batch_size=batch_size)

    return train_generator, val_generator, test_generator


In [22]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate= 0.0005),
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

In [ ]:
# Early stopping callback



early_stopping = callbacks.EarlyStopping(
    monitor='accuracy',  # Monitor validation loss
    patience=10,  # Stop training after 5 epochs without improvement
    restore_best_weights=True)  # Restore weights of the best epoch)

lr_callback = ReduceLROnPlateau(monitor='accuracy', factor=0.5, patience=5,
                                verbose=1, mode='auto', min_delta=0, cooldown=0, min_lr=0)

#train_generator = create_generators(train_data_split, train_label_split , validation_data_split,validation_label_split, 100,batch_size=32)
train_generator, val_generator, test_generator = create_generators(train_data,
                                                                    train_labels ,
                                                                      val_data,val_labels,
                                                                          test_data,
                                                                              test_labels,
                                                                                  100,
                                                                                      batch_size=32)


history = model.fit(train_generator, validation_data=val_generator,
                    steps_per_epoch=int((len(train_data))),
                    validation_steps=100,
                      epochs=100,
                      callbacks=[early_stopping, lr_callback],
                      verbose=1)




Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:593: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


552/552 ━━━━━━━━━━━━━━━━━━━━ 31s 31ms/step - accuracy: 0.6732 - loss: 8.5829 - val_accuracy: 0.6744 - val_loss: 1.3054 - learning_rate: 5.0000e-04
Epoch 2/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 14s 26ms/step - accuracy: 0.8107 - loss: 0.9489 - val_accuracy: 0.5306 - val_loss: 1.5571 - learning_rate: 5.0000e-04
Epoch 3/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 14s 26ms/step - accuracy: 0.8213 - loss: 0.6385 - val_accuracy: 0.6297 - val_loss: 0.8340 - learning_rate: 5.0000e-04
Epoch 4/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 15s 28ms/step - accuracy: 0.8336 - loss: 0.5559 - val_accuracy: 0.7800 - val_loss: 0.6201 - learning_rate: 5.0000e-04
Epoch 5/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 11s 20ms/step - accuracy: 0.8350 - loss: 0.5287 - val_accuracy: 0.7138 - val_loss: 0.7603 - learning_rate: 5.0000e-04
Epoch 6/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 13s 23ms/step - accuracy: 0.8414 - loss: 0.5041 - val_accuracy: 0.5100 - val_loss: 1.2393 - learning_rate: 5.0000e-04
Epoch 7/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 13s 23ms/step - a